In [1]:
from glob import glob
import pandas as pd
import os
import nbformat

In [2]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
len(book2), book2[0]

(183, '../../notebooks/book2/18/bnn_mnist_sgld.ipynb')

In [3]:
suffix = ["tfp", "blackjax", "jax", "pyro", "numpyro", "pymc3", "pymc", "sklearn", "flax"]
suffix = ["_" + suff for suff in suffix]  # add underscore

check_suffix = lambda nb_name: any([suff in nb_name for suff in suffix])
nb_suffix = list(filter(lambda x: check_suffix(x), book2))
len(nb_suffix), nb_suffix

(14,
 ['../../notebooks/book2/11/funnel_numpyro.ipynb',
  '../../notebooks/book2/11/mcmc_traceplots_unigauss_numpyro.ipynb',
  '../../notebooks/book2/17/gpc_demo_2d_sklearn.ipynb',
  '../../notebooks/book2/15/activation_fun_deriv_jax.ipynb',
  '../../notebooks/book2/15/lecun1989_flax.ipynb',
  '../../notebooks/book2/14/linreg_hierarchical_non_centered_blackjax.ipynb',
  '../../notebooks/book2/14/linreg_height_weight_numpyro.ipynb',
  '../../notebooks/book2/14/logreg_iris_bayes_2d_pymc3.ipynb',
  '../../notebooks/book2/20/vdvae_flax_demo_cifar.ipynb',
  '../../notebooks/book2/28/causal_impact_tfp.ipynb',
  '../../notebooks/book2/07/laplace_approx_beta_binom_jax.ipynb',
  '../../notebooks/book2/07/hmc_beta_binom_blackjax.ipynb',
  '../../notebooks/book2/07/advi_beta_binom_jax.ipynb',
  '../../notebooks/book2/22/flow_spline_mnist_jax.ipynb'])

In [4]:
def replace_to_without_suffix(nb_path, do_replace):
    old_path = nb_path
    for suff in suffix:
        nb_path = nb_path.replace(suff, "")

    if do_replace:
        os.rename(old_path, nb_path)
    return (old_path, nb_path)

In [5]:
nb_pairs = list(map(replace_to_without_suffix, nb_suffix, [1] * len(nb_suffix)))
len(nb_pairs), nb_pairs

(14,
 [('../../notebooks/book2/11/funnel_numpyro.ipynb',
   '../../notebooks/book2/11/funnel.ipynb'),
  ('../../notebooks/book2/11/mcmc_traceplots_unigauss_numpyro.ipynb',
   '../../notebooks/book2/11/mcmc_traceplots_unigauss.ipynb'),
  ('../../notebooks/book2/17/gpc_demo_2d_sklearn.ipynb',
   '../../notebooks/book2/17/gpc_demo_2d.ipynb'),
  ('../../notebooks/book2/15/activation_fun_deriv_jax.ipynb',
   '../../notebooks/book2/15/activation_fun_deriv.ipynb'),
  ('../../notebooks/book2/15/lecun1989_flax.ipynb',
   '../../notebooks/book2/15/lecun1989.ipynb'),
  ('../../notebooks/book2/14/linreg_hierarchical_non_centered_blackjax.ipynb',
   '../../notebooks/book2/14/linreg_hierarchical_non_centered.ipynb'),
  ('../../notebooks/book2/14/linreg_height_weight_numpyro.ipynb',
   '../../notebooks/book2/14/linreg_height_weight.ipynb'),
  ('../../notebooks/book2/14/logreg_iris_bayes_2d_pymc3.ipynb',
   '../../notebooks/book2/14/logreg_iris_bayes_2d.ipynb'),
  ('../../notebooks/book2/20/vdvae_flax

## Update latex source code

In [9]:
tex_files = glob("../../../bookv2/book2/*/*.tex")
tex_files[5], len(tex_files)

('../../../bookv2/book2/comp/comp-methods.tex', 384)

In [10]:
def make_latex_nb(nb_path):
    """
    foo_jax -> foo\_jax
    """
    nb = nb_path.split("/")[-1]
    nb = nb.replace("_", "\_").replace(".ipynb", "")
    return nb


def replace_keyword_in_tex(tex_file, old_nb, new_nb):

    with open(tex_file, "r") as fp:
        content = fp.read()

    old = make_latex_nb(old_nb)  # foo_jax -> foo\_jax
    new = make_latex_nb(new_nb)

    if old in content:
        print(f"In {tex_file} - {old} -> {new}")
    content = content.replace(old, new)

    with open(tex_file, "w") as fp:
        fp.write(content)

In [11]:
# replace_keyword_in_tex(tex_files[5],"advi_beta_binom_jax.ipynb","advi_beta_binom.ipynb")

In [12]:
for pair in nb_pairs:
    old, new = pair
    # print(old, new)
    list(map(replace_keyword_in_tex, tex_files, [old] * len(tex_files), [new] * len(tex_files)))

In ../../../bookv2/book2/gp/gp-classification.tex - gpc\_demo\_2d\_sklearn -> gpc\_demo\_2d
In ../../../bookv2/book2/dnn/blocks.tex - activation\_fun\_deriv\_jax -> activation\_fun\_deriv
In ../../../bookv2/book2/dnn/canonical.tex - lecun1989\_flax -> lecun1989
In ../../../bookv2/book2/glm/glmm.tex - linreg\_hierarchical\_non\_centered\_blackjax -> linreg\_hierarchical\_non\_centered
In ../../../bookv2/book2/glm/glm-linreg.tex - linreg\_height\_weight\_numpyro -> linreg\_height\_weight
In ../../../bookv2/book2/glm/logreg.tex - logreg\_iris\_bayes\_2d\_pymc3 -> logreg\_iris\_bayes\_2d
In ../../../bookv2/book2/nfm/hvae.tex - vdvae\_flax\_demo\_cifar -> vdvae\_demo\_cifar
In ../../../bookv2/book2/ssm/impact.tex - causal\_impact\_tfp -> causal\_impact
In ../../../bookv2/book2/misc/dci.tex - causal\_impact\_tfp -> causal\_impact
In ../../../bookv2/book2/comp/comp-methods.tex - laplace\_approx\_beta\_binom\_jax -> laplace\_approx\_beta\_binom
In ../../../bookv2/book2/comp/comp-methods.tex - 

## Update in notebooks 

In [13]:
def apply_fun_to_notebook(notebook, fun, old_nb_path, new_nb_path):
    """
    fun should take one argument: code
    """
    nb = nbformat.read(notebook, as_version=4)
    for cell in nb.cells:
        code = cell["source"]
        updated_code = fun(code, old_nb_path, new_nb_path)
        if updated_code != code:
            cell["source"] = updated_code
            nbformat.write(nb, notebook)


def replace_keyword(code, old_nb_path, new_nb_path):
    old_nb = old_nb_path.split("/")[-1]
    new_nb = new_nb_path.split("/")[-1]
    code = code.replace(old_nb, new_nb)
    return code

In [14]:
book2 = glob("../../notebooks/book2/*/*.ipynb")
for pair in nb_pairs:
    list(
        map(
            apply_fun_to_notebook, book2, [replace_keyword] * len(book2), [pair[0]] * len(book2), [pair[1]] * len(book2)
        )
    )